# Anomaly Detection: STEP2

## Outline
1. Introduction  
2. Import Module  
3. Load Dataset
    - 3.1 Description
    - 3.2 Difine Anomalies
4. Split Dataset(Train, Test)
5. Exploratory Data Analysis
    - 5.1 Distribution
    - 5.2 Correlation
6. Implementation Anomaly Detection

# Introduction
[UCI machine learning repository](https://archive.ics.uci.edu/ml/index.php)의 [Covertype Data Set](https://archive.ics.uci.edu/ml/datasets/Covertype)를 이용하여 Point Anomalies 탐지 구현을 목표로 하는 커널이다.

주어진 원 데이터는 581,012개의 행과 55개의 열로 구성되어 있으며, Target 변수를 제외하고 10개의 수치형 변수와 44개의 범주형 변수이다. 

Target 변수는 총 7 개의 값을 가지고 있다. 

이상 탐지 구현은 수치형 변수 10개만 가지고 수행할 것이다.

In [13]:
import pandas as pd
import numpy as np

import warnings
import os
warnings.filterwarnings('ignore')
print(os.listdir('../dataset/forest_cover_type'))

['covtype.csv']


# Load Data

In [25]:
data = pd.read_csv('../dataset/forest_cover_type/covtype.csv')
print("Data Set Shape: ", data.shape)

Data Set Shape:  (581012, 55)


In [26]:
data.head()

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,...,Soil_Type32,Soil_Type33,Soil_Type34,Soil_Type35,Soil_Type36,Soil_Type37,Soil_Type38,Soil_Type39,Soil_Type40,Cover_Type
0,2596,51,3,258,0,510,221,232,148,6279,...,0,0,0,0,0,0,0,0,0,5
1,2590,56,2,212,-6,390,220,235,151,6225,...,0,0,0,0,0,0,0,0,0,5
2,2804,139,9,268,65,3180,234,238,135,6121,...,0,0,0,0,0,0,0,0,0,2
3,2785,155,18,242,118,3090,238,238,122,6211,...,0,0,0,0,0,0,0,0,0,2
4,2595,45,2,153,-1,391,220,234,150,6172,...,0,0,0,0,0,0,0,0,0,5


## Description
구현에 사용할 변수는 다음과 같다.
- Elevation: 해발 고도
- Aspect: 방위각
- Slope: 경사 등급
- Horizontal_Distance_To_Hydrology: 가장 가까운 지표수까지의 수평 거리
- Vertical_Distance_To_Hydrology: 가장 가까운 지표수까지의 수직 거리
- Horizontal_Distance_To_Roadways: 가장 가까운 도로까지의 수평 거리
- Hillshade_9am (0 to 255 index): 여름철 오전 9시에 발생한 그늘의 정도
- Hillshade_Noon (0 to 255 index): 여름철 정오에 발생한 그늘의 정도
- Hillshade_3pm (0 to 255 index): 여름철 오후 3시에 발생한 그늘의 정도
- Horizontal_Distance_To_Fire_Points: 가장 가까운 산불 발화지점까지의 수평 거리

In [29]:
label = data['Cover_Type']

data = data.iloc[:, :10]
data['Cover_Type'] = label

data.describe()

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,Cover_Type
count,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000
mean,2959.365301,155.656807,14.103704,269.428217,46.418855,2350.146611,212.146049,223.318716,142.528263,1980.291226,2.051471
std,279.984734,111.913721,7.488242,212.549356,58.295232,1559.254870,26.769889,19.768697,38.274529,1324.195210,1.396504
min,1859.000000,0.000000,0.000000,0.000000,-173.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
25%,2809.000000,58.000000,9.000000,108.000000,7.000000,1106.000000,198.000000,213.000000,119.000000,1024.000000,1.000000
50%,2996.000000,127.000000,13.000000,218.000000,30.000000,1997.000000,218.000000,226.000000,143.000000,1710.000000,2.000000
75%,3163.000000,260.000000,18.000000,384.000000,69.000000,3328.000000,231.000000,237.000000,168.000000,2550.000000,2.000000
max,3858.000000,360.000000,66.000000,1397.000000,601.000000,7117.000000,254.000000,254.000000,254.000000,7173.000000,7.000000


## Define Anomalies
Target 변수는 7개의 값으로 이루어져 있다. 값에 대한 설명은 다음과 같다.
- 1 = <b>Spruce/Fir</b>: 전나무로 불린다.

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/4/47/Black_spruce_stand_at_Arctic_Chalet%2C_Inuvik%2C_NT.jpg/330px-Black_spruce_stand_at_Arctic_Chalet%2C_Inuvik%2C_NT.jpg" width = "250">


- 2 = <b>Lodgepole Pine</b>: 로지폴 소나무로 불린다. 미 서부에 서식하며, 해안 근처와 고산대까지는 많이 관측되지만 저지대에서는 드물다는 특징을 보인다.

<img src="https://www.for.gov.bc.ca/hfd/library/documents/treebook/lodgepolepine.jpg" width = "250">

- 3 = <b>Ponderosa Pine</b>: 폰데로사 소나무로 불린다. 미 서부의 산악지역에 주로 서식하며 이외에도 다양한 곳에서 서식한다.

<img src="https://i.pinimg.com/564x/e9/09/34/e909349f0067010243f448bc8620db2b.jpg" width = "250">

- 4 = <b>Cottonwood/Willow</b>: 사시나무로 불린다. 온대 기후에서 서식한다. 영양 생식을 하기 때문에, 많은 잡종이 생긴다는 특징이 있다.

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/1/16/Populier_Populus_canadensis.jpg/375px-Populier_Populus_canadensis.jpg" width = "250">

- 5 = <b>Aspen</b>: 아스펜이라고 불린다. 여름이 서늘하고 산이나 고원과 같은 높은 고도에서 주로 서식한다. 일반적으로 침엽수 나무가 서식하는 환경에서 자란다.

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/2/29/Snowbowlaspens.jpg/330px-Snowbowlaspens.jpg" width = "250">

- 6 = <b>Douglas-fir</b>: 소나무 과에 속하는 상록수이다. 

<img src="https://cdn.britannica.com/s:700x500/34/118234-050-12C9BAC8/Douglas-fir.jpg" width = "250">

- 7 = <b>Krummholz</b> : 아열대 및 아열 고산에서 나타나는 변종 식물의 일종이다. 매우 추운 바람에 지속적으로 노출되어 나타난다.

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/d/d7/Pinus_albicaulis_7872.JPG/330px-Pinus_albicaulis_7872.JPG" width = "250">



In [30]:
data['Cover_Type'].value_counts()

2    283301
1    211840
3     35754
7     20510
6     17367
5      9493
4      2747
Name: Cover_Type, dtype: int64